In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import requests
import random

from rtree import index
from shapely.geometry import box
from utils import *

In [2]:
G = ox.save_load.load_graphml(filename="Central_SG_drive_processed.graphml")
G_ped = ox.save_load.load_graphml(filename="Central_SG_walk.graphml")

# Stores nodes and edges mapped with speed bands along with other attributes
print(type(G))
print(type(G_ped))

<class 'networkx.classes.multidigraph.MultiDiGraph'>
<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [3]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [4]:
f = open("../../Traffic speed bands/Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)

/home/orionstar25/Semester-8-Major-Project/Open Street Maps/Phase 5/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


In [5]:
calculate_congestion(G, edges, speed_bands)

In [6]:
# ox.save_load.save_graph_shapefile(G, filename='Central_SG')

In [7]:
# Filter out all bus stop nodes
# key: node label ,value: location
idx = index.Index()

xx = nodes['x'].astype(float)
yy = nodes['y'].astype(float)
node_id = nodes['osmid']

for node, x, y in zip(node_id, xx, yy):
    idx.insert(int(node), (x,y,x,y))

### Middle mile routing

In [8]:
def modified_hybrid_search(G, x):
    n = len(x)
    cutoff = math.floor(n/math.exp(1))
    min_pair = x[0]
    source = min_pair[0][0]
    dest = min_pair[0][1]
    min_tt = float('inf')

    for i in range(n):
        try:            
            travel_time = nx.shortest_path_length(G, source, dest, weight='BPR', method='dijkstra')
        except nx.NetworkXUnfeasible:
            print("tch")
            continue

        # have reached cutoff and didnt find optimal yet
        if i > cutoff:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]
                break
            else:
                break
        else:
            if travel_time < min_tt:
                min_tt = travel_time
                min_pair = x[i]

                if i < n-1:
                    if travel_time < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break
            else:
                if i < n-1:
                    if min_tt < x[i+1][1]:
                        break
                    else:
                        source = x[i+1][0][0]
                        dest = x[i+1][0][1]
                else:
                    break

    return min_tt, min_pair[0]

In [10]:
# get 10 random sources
sources = []
for i in range(10):
    r = random.randint(0,5640)
    s = nodes['osmid'].values[r]
    sources.append(s)

# get 10 correspondig random targets
targets = []
for i in range(20):
    r = random.randint(0,5640)
    t = nodes['osmid'].values[r]
    targets.append(t)
    
AD = []
demand = []

for s in sources:
    for t in targets:
        print(s, t)
        AD.append([s,t])
        orig_point = nodes[nodes['osmid'] == s].geometry.values[0]
        target_point = nodes[nodes['osmid'] == t].geometry.values[0]
        orig_xy = (orig_point.y, orig_point.x)
        target_xy = (target_point.y, target_point.x)

        # Max distance for walking is 720m (10 min * 1.2 m/s)
        # Find all nodes within 720 m from candidate sources and candidate destinations
        candidate_dest = find_nearest_nodes(nodes, target_xy, 400, idx)
        candidate_source = find_nearest_nodes(nodes, orig_xy, 400, idx)

        if len(candidate_dest) == 0:
            candidate_dest.append(ox.get_nearest_node(G, target_xy))

        if len(candidate_source) == 0:
            candidate_source.append(ox.get_nearest_node(G, orig_xy))

        euclidean = get_length_dict(nodes, candidate_source, candidate_dest)
        _, OD = modified_hybrid_search(G, euclidean)
        demand.append(OD)

425482453 4744279229


ValueError: ('Contradictory paths found:', 'negative weights?')

## First mile

In [12]:
# use pedestrain network only----> change this
def print_route(route):
  i = 0
  n = len(route)

  for i in range(n):
    for nei,w in G_whole[route[i]].items():
      if i+1 != n:
        if nei == route[i+1]:
          print(w[0].get('highway'), w[0].get('length'))

In [31]:
orig_point = nodes[nodes['osmid'] == AD[0][0]].geometry.values[0]
orig_xy = (orig_point.y, orig_point.x)
orig_node, dist = ox.get_nearest_node(G_ped, orig_xy, method='haversine', return_dist = True)

target_point = nodes[nodes['osmid'] == demand[0][0]].geometry.values[0]
target_xy = (target_point.y, target_point.x)
target_node = demand[0][0]

first_mile_length, first_mile = nx.single_source_dijkstra(G_ped, orig_node, target_node, weight='length')

print_route(first_mile)
print("Distance from X to A: ", dist) # in metres
print("Distance from A to B: ", first_mile_length) # in metres

residential 102.051
residential 7.747
residential 32.174
residential 124.57300000000001
residential 56.33
Distance from X to A:  482.23028553412615
Distance from A to B:  322.875


### Last Mile

In [32]:
target_node, dist = ox.get_nearest_node(G_whole, target_xy, method='haversine', return_dist= True)
last_mile = nx.shortest_path(G_whole, sd_pair[0][1], target_node, weight='length', method='dijkstra')
last_mile_length = nx.shortest_path_length(G_whole, sd_pair[0][1], target_node, weight='length', method='dijkstra')

print_route(last_mile)
print("Distance from C to D: ", last_mile_length)
print("Distance from D to Y: ", dist)

primary 561.0409999999999
primary 31.486
primary 10.98
primary 17.553
primary 11.666
primary 26.612
primary 256.842
primary 9.377
tertiary 13.573
primary 8.92
Distance from C to D:  948.0499999999998
Distance from D to Y:  0.0
